yahb - yet another hybrid bench

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np

from torchtext.legacy.data import Field, TabularDataset, BucketIterator

from dataset.mtgcards import RuleText
from utils.preprocess import fields_for_rule_text

import random
from tqdm import tqdm


d:\Miniconda3\envs\seq2seq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SRC, TRG = fields_for_rule_text(include_lengths=False, batch_first=True)
fields = {'src': ('src', SRC), 'trg': ('trg', TRG)}

train_data, valid_data, test_data = RuleText.splits(fields=fields, version='v2.2')

In [3]:
SRC.build_vocab(train_data, min_freq = 4)
TRG.build_vocab(train_data, min_freq = 4)
print(f"Unique tokens in source (en) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (zh) vocabulary: {len(TRG.vocab)}")

for x in random.sample(list(train_data), 1):
    print(x.src, x.trg)

Unique tokens in source (en) vocabulary: 1294
Unique tokens in target (zh) vocabulary: 1949
['{', '1', '}', '{', 'r', '}', ',', 'remove', 'a', 'time', 'counter', 'from', 'a', '<', '0', '>', 'you', 'control', 'or', 'suspended', 'card', 'you', 'own', ':', '<', '9', '>', 'gets', '+', '2', '/', '+', '0', 'until', 'end', 'of', 'turn', '.'] ['{', '1', '}', '{', 'r', '}', '，', '从', '由', '你', '操控', '的', '<', '0', '>', '或', '由', '你', '拥有', '且', '已', '延缓', '的', '牌上', '移', '去', '一个', '计时', '指示', '物', '：', '<', '9', '>', '得', '+', '2', '/', '+', '0', '直到', '回合', '结束', '。']


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_within_batch = True,
    sort_key = lambda x: len(x.src),
    device = device)

print(next(iter(train_iterator)))

cpu

[torchtext.legacy.data.batch.Batch of size 128]
	[.src]:[torch.LongTensor of size 128x4]
	[.trg]:[torch.LongTensor of size 128x11]


In [5]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        #self.pos_embedding = nn.Embedding(max_length, hid_dim)
        self.pos_embedding = PositionalEncoding(hid_dim, max_length)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        src = self.dropout(self.pos_embedding(self.tok_embedding(src) * self.scale))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src
    

class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src
    

class PositionalEncoding(nn.Module):

    def __init__(self, d_hid, n_position=200):
        super(PositionalEncoding, self).__init__()

        # Not a parameter
        self.register_buffer('pos_table', self._get_sinusoid_encoding_table(n_position, d_hid))

    def _get_sinusoid_encoding_table(self, n_position, d_hid):
        ''' Sinusoid position encoding table '''
        # TODO: make it with torch instead of numpy

        def get_position_angle_vec(position):
            return [position / np.power(10000, 2 * (hid_j // 2) / d_hid) for hid_j in range(d_hid)]

        sinusoid_table = np.array([get_position_angle_vec(pos_i) for pos_i in range(n_position)])
        sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
        sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1

        return torch.FloatTensor(sinusoid_table).unsqueeze(0)

    def forward(self, x):
        return x + self.pos_table[:, :x.size(1)].clone().detach()
    

class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention
    

class PositionwiseFeedforwardLayer(nn.Module):
    
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [6]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear(enc_hid_dim + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs, mask):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
  
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention = [batch size, src len]
        
        attention = attention.masked_fill(mask == 0, -1e10)
        
        return F.softmax(attention, dim = 1)
    

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(enc_hid_dim + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear(enc_hid_dim + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs, mask):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        #mask = [batch size, src len]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs, mask)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

In [7]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]

        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        src_mask = self.make_src_mask(src)
        #src_mask = [batch size, 1, 1, src len]

        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]

        enc_src = enc_src.permute(1, 0, 2).contiguous()
        #enc_src = [src len, batch size, enc hid dim * 2]
        
        trg = trg.permute(1, 0).contiguous()
        #trg = [trg len, batch size]

        #first input to the decoder is the <sos> tokens
        input = trg[0,:].contiguous()

        hidden = enc_src[0,:,:]
        #hidden = [batch size, dec hid dim]

        mask=src_mask.squeeze()
        #mask = [batch size, src len]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state, all encoder hidden states 
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(input, hidden, enc_src, mask)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
            
        return outputs

In [8]:
import torch
import torch.nn as nn

from tqdm import tqdm


def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)


def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    # for i, batch in enumerate(iterator):
    for i, batch in tqdm(enumerate(iterator), total=len(iterator)):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        trg = trg.permute(1, 0)
                
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]

        output_dim = output.shape[-1]
            
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].contiguous().view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
            
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)


def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg
            
            output = model(src, trg)
            trg = trg.permute(1, 0)

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
                
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].contiguous().view(-1)
            
            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]
                
            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
        
    return epoch_loss / len(iterator)

In [9]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
# HID_DIM = 256
HID_DIM = 512
ENC_LAYERS = 3
ENC_HEADS = 8
ENC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

attn = Attention(HID_DIM, HID_DIM)
enc = Encoder(INPUT_DIM, 
            HID_DIM, 
            ENC_LAYERS, 
            ENC_HEADS, 
            ENC_PF_DIM, 
            ENC_DROPOUT, 
            device)

dec = Decoder(OUTPUT_DIM, 
            HID_DIM, 
            HID_DIM, 
            HID_DIM, 
            DEC_DROPOUT, 
            attn)
model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [10]:
from utils import count_parameters, train_loop
model.apply(initialize_weights)
count_parameters(model)

12277661

In [22]:
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
train_loop(model, optimizer, criterion, train, evaluate,
           train_iterator, valid_iterator, 
           save_path='result/', file_name='hybrid-model-rule-v2.2.pt', load_before_train=True)

model will be saved to result/hybrid-model-rule-v2.2.pt


100%|██████████| 454/454 [00:43<00:00, 10.47it/s]


Epoch: 01 | Time: 0m 44s
	Train Loss: 2.418 | Train PPL:  11.228
	 Val. Loss: 1.227 |  Val. PPL:   3.412


100%|██████████| 454/454 [00:41<00:00, 10.89it/s]


Epoch: 02 | Time: 0m 42s
	Train Loss: 0.923 | Train PPL:   2.516
	 Val. Loss: 0.700 |  Val. PPL:   2.014


...

Epoch: 08 | Time: 0m 42s
	Train Loss: 0.313 | Train PPL:   1.368
	 Val. Loss: 0.324 |  Val. PPL:   1.383


100%|██████████| 454/454 [00:41<00:00, 11.04it/s]


Epoch: 09 | Time: 0m 41s
	Train Loss: 0.289 | Train PPL:   1.335
	 Val. Loss: 0.325 |  Val. PPL:   1.384


100%|██████████| 454/454 [00:41<00:00, 10.93it/s]


Epoch: 10 | Time: 0m 42s
	Train Loss: 0.262 | Train PPL:   1.300
	 Val. Loss: 0.331 |  Val. PPL:   1.392


In [11]:
import math
def tok_cat(toks: list, delim: str = '')->str:
    s = ''
    for t in toks:
        s += t + delim
    return s

def beam_search(sentence, src_field, trg_field, model, device, max_len = 50, beam_size=3, print_per_step=False, branch_size=None):
    if branch_size is None:
        branch_size = beam_size
    
    model.eval()
        
    if isinstance(sentence, str):
        nlp = src_field.tokenize
        tokens = [token.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    src_len = torch.LongTensor([len(src_indexes)]).to(device)
    
    with torch.no_grad():
        encoder_outputs = model.encoder(src_tensor, src_len).permute(1, 0, 2)
        hidden = encoder_outputs[0,:,:]

    mask = model.make_src_mask(src_tensor).squeeze().to(device)
        
    attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)

    trg_indexes = [(.0, [trg_field.vocab.stoi[trg_field.init_token]], hidden, attentions)]
    
    for i in range(max_len):

        candidates = []
        terminate = True
        for j in range(len(trg_indexes)):
            
            cur_prob, cur_list, cur_hid, cur_att = trg_indexes[j]
            if cur_list[-1] == trg_field.vocab.stoi[trg_field.eos_token]:
                candidates.append(trg_indexes[j])
                continue
            
            terminate = False

            trg_tensor = torch.LongTensor([cur_list[-1]]).to(device)
                    
            with torch.no_grad():
                output, hidden, attention = model.decoder(trg_tensor, cur_hid, encoder_outputs, mask)
                output = F.softmax(output, dim=1)
            cur_attentions = cur_att.clone()
            cur_attentions[i] = attention

            prob_tokens, pos_tokens = torch.topk(output, branch_size, dim=1)
            for prob, pos in zip(prob_tokens[0], pos_tokens[0]):
                candidates.append((cur_prob + math.log(prob), cur_list + [pos], hidden, cur_attentions))
        
        if terminate:
            break

        candidates.sort(reverse=True, key = lambda x: x[0])
        trg_indexes = candidates[:beam_size]
        if print_per_step:
            trg_tokens = [[trg_field.vocab.itos[i] for i in id_list[1:]] for _, id_list, __, ___ in trg_indexes]
            print(f'candidates of step {i}')
            print([tok_cat(t) for t in trg_tokens])
            print([math.exp(p) for p, _, _, _ in trg_indexes])


    trg_tokens = [[trg_field.vocab.itos[i] for i in id_list[1:]] for _, id_list, _, _ in trg_indexes]
    probs = [math.exp(p) for p, _, _, _ in trg_indexes]
    atts = [att[:len(tok_list)-1] for _, tok_list, _, att in trg_indexes]

    return trg_tokens, probs, atts

In [12]:
from utils.translate import Translator
model.load_state_dict(torch.load('result/hybrid-model-rule-v2.2.pt', map_location=torch.device(device)))
T = Translator(SRC, TRG, model, device, beam_search)

In [13]:
data = 'Whenever <1> becomes attached to a creature, for as long as <1> remains attached to it, you may have that creature become a copy of another target creature you control.'
data = 'target creature gets - 1 / - 1 until end of turn .'
ret, prob = T.translate(data, max_len=100)
print(*ret[:3], sep='\n')

['目标', '生物', '得', '-', '1', '/', '-', '1', '直到', '回合', '结束', '。', '<eos>']
['目标', '生物', '得', '-', '1', '1', '/', '-', '1', '直到', '回合', '结束', '。', '<eos>']
['目标', '生物', '得', '+', '1', '/', '-', '1', '直到', '回合', '结束', '。', '<eos>']


In [14]:
from dataset.mtgcards import TestSets
from utils import calculate_bleu
from torchtext.legacy.data import Field
from models.card_name_detector.definition import TrainedDetector
from utils.translate import sentencize, CardTranslator, CTHelper

fields = {'src-rule': ('src', Field(tokenize=lambda x: x.split(' '))), 'trg-rule': ('trg', Field())}
test_data = TestSets.load(fields)

D = TrainedDetector()

path: d:\Desktop\mtg-cards-translation\models\card_name_detector


In [16]:
dic = {}
dic = {'oil':'烁油', 'rebel':'反抗军'}
helper = CTHelper(D, dic)
CT = CardTranslator(sentencize, T, 
                    preprocess=lambda x: helper.preprocess(x, False), 
                    postprocess=lambda x: helper.postprocess(x, False))

example = list(test_data)[13]
example = list(test_data)[8]
# ret = CT.translate(' '.join(example.src))
# print(ret)
for example in random.sample(list(test_data), 3):
    print(vars(example))
    ret = CT.translate(' '.join(example.src))
    print(ret)

{'src': ['Hexproof\n+2:', 'Each', 'opponent', 'loses', '3', 'life', 'and', 'you', 'gain', '3', 'life.\n0:', 'You', 'draw', 'two', 'cards.', 'Then', 'each', 'opponent', 'may', 'scry', '1.\n−3:', 'Exile', 'target', 'creature', 'or', 'enchantment.', 'If', 'it', "wasn't", 'an', 'Aura,', 'create', 'a', 'token', "that's", 'a', 'copy', 'of', 'it,', 'except', "it's", 'a', '1/1', 'white', 'Spirit', 'creature', 'with', 'flying', 'in', 'addition', 'to', 'its', 'other', 'types.'], 'trg': ['辟邪', '+2：每位对手各失去3点生命且你获得3点生命。', '0：你抓两张牌。然后每位对手各可以占卜1。', '−3：放逐目标生物或结界。如果它不是灵气，则派出一个为其复制品的衍生物，但它是1/1白色，具飞行异能的精怪生物，且仍具有原本类别。']}
[after preprocess]:hexproof
[before postprocess]:辟邪
[after preprocess]:+ 2 : each opponent loses 3 life and you gain 3 life .
[before postprocess]:+2：每位对手各失去3点生命且你获得3点生命。
[after preprocess]:0 : you draw two cards .
[before postprocess]:0：你抓两张牌。
[after preprocess]:then each opponent may scry 1 .
[before postprocess]:然后每位对手各可以占卜1。
[after preprocess]:− 3 : exile target creature or enchantme

In [17]:
from utils import calculate_testset_bleu
calculate_testset_bleu(list(test_data)[:100], CT)

  0%|          | 0/100 [00:00<?, ?it/s]

[after preprocess]:whenever another artifact or creature you control is put into a graveyard from the battlefield , put an <1> counter on <0> .
[before postprocess]:每当另一个由你操控的神器或生物从战场进入坟墓场时，在<1>上放置一个<1>指示物。
[after preprocess]:{2} {b} , {t} , remove four <1> counters from <0> : return target creature card from your graveyard to the battlefield .


  1%|          | 1/100 [00:00<00:47,  2.07it/s]

[before postprocess]:{2}{b}，{t}，从<0>上移去四个充电指示物：将目标生物牌从你的坟墓场移回战场。
[after preprocess]:activate only as a sorcery .
[before postprocess]:只可以于你能施放法术的时机下起动此异能。
[after preprocess]:vigilance
[before postprocess]:警戒
[after preprocess]:<0> enters the battlefield with an <1> counter on it .
[before postprocess]:<0>进战场时上面有一个<1>指示物。
[after preprocess]:remove an <1> counter from <0> : it becomes an artifact creature until end of turn .
[before postprocess]:从<1>上移去一个<1>：它成为成为神器生物直到回合结束。
[after preprocess]:crew 3
[before postprocess]:搭载3
[after preprocess]:tap any number of creatures you control with total power 3 or more : this vehicle becomes an artifact creature until end of turn .


  2%|▏         | 2/100 [00:00<00:46,  2.12it/s]

...

 99%|█████████▉| 99/100 [00:33<00:00,  3.35it/s]

[before postprocess]:{1}{g}，{t}，牺牲<0>：抓一张牌。
[after preprocess]:choose one —
[before postprocess]:选择一项～
[after preprocess]:• each opponent sacrifices a non token creature .
[before postprocess]:•每位对手各牺牲一个非衍生物的生物。
[after preprocess]:• each opponent sacrifices a creature token .


100%|██████████| 100/100 [00:33<00:00,  2.96it/s]

[before postprocess]:•每位对手各牺牲一个生物。
[after preprocess]:• each opponent sacrifices a planeswalker .
[before postprocess]:•每位对手各牺牲一个鹏洛客。


0.6673989994232405